In [2]:
# Import packages
## Prepare and explore data
import pandas as pd # Data manipulation
import numpy as np # Numeric computations
import matplotlib.pyplot as plt # Data visualisation
import seaborn as sns # Data visualisation

## Machine learning packages
from sklearn.model_selection import train_test_split # Split training and test data
from sklearn.metrics import mean_absolute_error, mean_squared_error # Model evaluation
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet #Linear models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor # Ensemble models
from sklearn.svm import SVC # Support vector machine
from sklearn.naive_bayes import GaussianNB # Naive Bayes
from sklearn.tree import DecisionTreeRegressor # Decision Tree
from xgboost import XGBRegressor # Extreme gradient boostng
from category_encoders import OneHotEncoder # One hot encoder
from sklearn.pipeline import Pipeline, make_pipeline # Pipeline

# Prepare Data
## Import 

In [4]:
# Import dataset
df = pd.read_csv("data/height_prediction.csv")

## Explore

In [5]:
# Dimensions of the dataset
df.shape

(371, 4)

In [7]:
# Dataset structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          371 non-null    int64  
 1   gender       371 non-null    object 
 2   mean_ulna    371 non-null    float64
 3   mean_height  371 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 11.7+ KB


In [8]:
# View first five rows
df.head()

,age,gender,mean_ulna,mean_height
0,51,male,29.0,164.3
1,53,male,29.0,168.1
2,69,female,24.0,161.0
3,57,male,29.0,164.1
4,66,male,31.0,171.0


## Split

In [15]:
# Specify features
features = ["age", "gender", "mean_ulna"]

# Specify target vector
target = "mean_height"

# Subset features
X = df[features]

# Subset target
y = df[target]

# Split dataset into 80% training and 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

# Inspect split
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(296, 3)
(75, 3)
(296,)
(75,)


In [16]:
y_train.head()

192    163.0
75     157.8
84     152.1
359    156.0
16     164.0
Name: mean_height, dtype: float64

# Build model
## Baseline

In [12]:
# Mean of the target
y_mean = y_train.mean()

# Generate baseline predictions
y_pred_baseline = [y_mean] * len(y_train)

# Evaluate baseline predictions
## Mean absolute error
mae_baseline = mean_absolute_error(y_train, y_pred_baseline)
print("Baseline mean absolute error", mae_baseline)

## Mean squared error
mse_baseline = mean_squared_error(y_train, y_pred_baseline)
print("Baseline mean squared error", mse_baseline)

## Root mean squared error
rmse_baseline = np.sqrt(mean_squared_error(y_train, y_pred_baseline))
print("Baseline root mean squared error", rmse_baseline)

TypeError: Could not convert ['femalemalefemalefemalefemalefemalefemalemalemalefemalefemalefemalefemalefemalefemalefemalemalefemalefemalefemalefemalefemalefemalemalemalefemalefemalefemalefemalefemalefemalemalefemalefemalefemalemalemalemalefemalemalefemalefemalefemalefemalefemalefemalefemalemalefemalefemalemalemalemalefemalefemalefemalemalemalefemalefemalemalefemalefemalemalefemalefemalefemalefemalefemalefemalemalemalefemalefemalemale'] to numeric

## Iterate

# Evaluate

# Communicate